In [1]:
import glob
import json
import os

from instagrapi import Client

from instaduler.const import *
from instaduler.model import Post, PostList

In [2]:
QUERY = 'love'
ID = 3561152360807760
TRACK_HIGHLIGHT_START_TIMES_IN_MS = [100]
FOLDER = ''
TEXT = ''

In [3]:
client = Client()
d = client.load_settings('config/session.json')

In [4]:
from instagrapi.extractors import extract_track


def custom_search_music(client: Client, query: str):
    params = {
        "query": query,
        "browse_session_id": client.generate_uuid(),
    }
    result = client.private_request("music/audio_global_search/", params=params)
    items = result['items']
    for item in items:
        item['track']['territory_validity_periods'] = {}
    return [extract_track(item["track"]) for item in result["items"]]


tracks = custom_search_music(client, QUERY)
tracks

[Track(id='3561152360807760', title='Love Lee', subtitle='', display_artist='AKMU', audio_cluster_id=291891636753301, artist_id=None, cover_artwork_uri=HttpUrl('https://cdn.fbsbx.com/v/t65.14500-21/368786383_2652734344877616_1338678711065079745_n.jpg?stp=dst-jpg_p526x296&_nc_cat=1&ccb=1-7&_nc_sid=c499f9&_nc_ohc=1RePfIWzp4YAX8F6dAQ&_nc_ht=cdn.fbsbx.com&oh=00_AfCwuYlMdkZwlCvCwva2Gykye6dsPtkJ7TiMDeqFcGGdWg&oe=6549E398', ), cover_artwork_thumbnail_uri=HttpUrl('https://cdn.fbsbx.com/v/t65.14500-21/368786383_2652734344877616_1338678711065079745_n.jpg?stp=dst-jpg_s168x128&_nc_cat=1&ccb=1-7&_nc_sid=c499f9&_nc_ohc=1RePfIWzp4YAX8F6dAQ&_nc_ht=cdn.fbsbx.com&oh=00_AfDySg2XDVIilu0MZrJ7J4yI2MMLhYJO4V8wCWPdomK_bw&oe=6549E398', ), progressive_download_url=HttpUrl('https://scontent.frmq2-2.fna.fbcdn.net/v/t39.12897-6/369692631_1855040948225056_4071911656672402959_n.m4a?_nc_cat=1&ccb=1-7&_nc_sid=c2de2f&_nc_ohc=vspBFP0SYBUAX-Nma8w&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.frmq2-2.fna&oh=00_AfA7c_eG2wY5a5TK_84O

In [5]:
track_to_post = None
for track in tracks:
    if track.audio_asset_id != ID:
        continue

    track.highlight_start_times_in_ms = TRACK_HIGHLIGHT_START_TIMES_IN_MS if len(
        TRACK_HIGHLIGHT_START_TIMES_IN_MS) > 0 else track.highlight_start_times_in_ms
    track_to_post = track
    break
track_to_post

Track(id='3561152360807760', title='Love Lee', subtitle='', display_artist='AKMU', audio_cluster_id=291891636753301, artist_id=None, cover_artwork_uri=HttpUrl('https://cdn.fbsbx.com/v/t65.14500-21/368786383_2652734344877616_1338678711065079745_n.jpg?stp=dst-jpg_p526x296&_nc_cat=1&ccb=1-7&_nc_sid=c499f9&_nc_ohc=1RePfIWzp4YAX8F6dAQ&_nc_ht=cdn.fbsbx.com&oh=00_AfCwuYlMdkZwlCvCwva2Gykye6dsPtkJ7TiMDeqFcGGdWg&oe=6549E398', ), cover_artwork_thumbnail_uri=HttpUrl('https://cdn.fbsbx.com/v/t65.14500-21/368786383_2652734344877616_1338678711065079745_n.jpg?stp=dst-jpg_s168x128&_nc_cat=1&ccb=1-7&_nc_sid=c499f9&_nc_ohc=1RePfIWzp4YAX8F6dAQ&_nc_ht=cdn.fbsbx.com&oh=00_AfDySg2XDVIilu0MZrJ7J4yI2MMLhYJO4V8wCWPdomK_bw&oe=6549E398', ), progressive_download_url=HttpUrl('https://scontent.frmq2-2.fna.fbcdn.net/v/t39.12897-6/369692631_1855040948225056_4071911656672402959_n.m4a?_nc_cat=1&ccb=1-7&_nc_sid=c2de2f&_nc_ohc=vspBFP0SYBUAX-Nma8w&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.frmq2-2.fna&oh=00_AfA7c_eG2wY5a5TK_84O8

In [ ]:
files = glob.glob(os.path.join(FOLDER, "*.mp4"))
try:
    post = Post(files=sorted(files), caption=TEXT)
except:
    pass

try:
    postlist = PostList.parse_file(SCHEDULE_FILE)
except:
    postlist = PostList()

try:
    if not isinstance(postlist.posts, list):
        postlist.posts = []

    postlist.posts.append(post)
    with open(SCHEDULE_FILE, 'w', encoding='utf-8') as file:
        json.dump(postlist.dict(), file, ensure_ascii=False, indent=4)
    print(f"Post from folder: {FOLDER}, with caption: {TEXT}")
except:
    print(f"An exception occurred while saving the JSON file.")